In [1]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [2]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Musical_Instruments_5.json.gz')

In [3]:
df = df.loc[:,['reviewText','overall']]

In [4]:
import numpy as np
df['sentiment'] = np.where(df['overall'] >= 4, 1, 0)

In [5]:
df = df.drop(['overall'],axis=1)

In [6]:
df.columns = ['review','sentiment']

In [7]:
df.to_csv('reviews_Musical_Instruments.csv', index=False)

### 데이터 전처리

In [8]:
import pandas as pd
import numpy as np

In [9]:
nsmc = pd.read_csv('reviews_Musical_Instruments.csv')

In [10]:
nsmc = nsmc.dropna()

In [11]:
nsmc.shape

(10254, 2)

In [12]:
with open('nsmc2.txt', 'w', encoding='utf8') as f:  
    f.write('\n'.join(nsmc['review']))  

In [13]:
from sentencepiece import SentencePieceTrainer
SentencePieceTrainer.Train('--input=nsmc2.txt --model_prefix=nsmc2 --vocab_size=3000')  # 총 어휘수 3000개

True

In [14]:
from sentencepiece import SentencePieceProcessor
sp = SentencePieceProcessor()
sp.Load("nsmc2.model")

True

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
cv = CountVectorizer(lowercase=False, tokenizer=sp.encode_as_pieces)

In [17]:
tdm = cv.fit_transform(nsmc['review'])

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x = tdm
y = nsmc['sentiment']

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [21]:
import joblib

In [22]:
cv.tokenizer = None

In [23]:
joblib.dump((cv,x_train,x_test,y_train,y_test), 'nsmc2.pkl')

['nsmc2.pkl']